# End to End Example in Pythia

## 1. Load Model, Tokenizer

In [1]:
import transformers
from transformers import AutoTokenizer, AutoConfig, pipeline, AutoModelForCausalLM
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from datasets import load_dataset
import torch.nn.functional as F
import gc

CACHE_DIR = "/home/daniel/.huggingface/"

In [2]:
model_name_or_path = "EleutherAI/pythia-70m"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", cache_dir=CACHE_DIR)
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left", legacy=False, cache_dir=CACHE_DIR)
tokenizer.pad_token_id = 0 
# clear the gpu memory
torch.cuda.empty_cache()
model.to(torch.device("cuda"))
print(model.device)

cuda:0


In [3]:
print(model)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

## 2. Run Supervised Fine-Tuning

### 2.1 Load Dataset

In [4]:
from datasets import load_dataset, load_metric

In [5]:
from typing import List, Dict, Iterable
import numpy as np

PromptResponseDataset = Iterable[Dict[str, str]]

def sanity_check_prompt_response_dataset(dataset: PromptResponseDataset):
    indexes = np.random.randint(0, len(dataset), 10)
    for idx in indexes:
        idx = int(idx)
        data = dataset[idx]
        assert "prompt" in data
        assert "response" in data
        assert isinstance(data["prompt"], str)
        assert isinstance(data["response"], str)

In [6]:
dataset = load_dataset("truthful_qa", "generation")["validation"]
# Make it into prompt-response format
dataset = dataset.map(lambda x: {'prompt': x['question'], 'response': x['best_answer']})
sanity_check_prompt_response_dataset(dataset)


In [7]:
dataset

Dataset({
    features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source', 'prompt', 'response'],
    num_rows: 817
})

Print some dataset statistics

In [8]:
# Print the max length of prompt + response
def get_max_length(dataset: PromptResponseDataset):
    # Get the max length of prompt + response
    max_length = 0
    for i in range(len(dataset)):
        prompt, response = dataset[i]['prompt'], dataset[i]['response']
        max_length = max(max_length, len(prompt) + len(response))
        
    return max_length

print(get_max_length(dataset))

334


In [9]:
# TODO: May need to truncate if SFT exceeds context length? 
# I'm not gonna worry about this for now. 

### 2.2 Tokenization

In [10]:
for i in range(1):
    example = dataset[i]
    print(example["prompt"])
    print(example["response"])
    output = tokenizer(example["prompt"], example["response"])
    print(output.keys())
    tokens = output['input_ids']
    masks = output['attention_mask']
    print(len(tokens))
    print(masks)
    # for token in tokens:
    #     print(tokenizer.decode(token))

What happens to you if you eat watermelon seeds?
The watermelon seeds pass through your digestive system
dict_keys(['input_ids', 'attention_mask'])
20
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [11]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["prompt"], 
        examples["response"]
    )
    return tokenized_examples

In [12]:
from datasets import DatasetDict

orig_col_names = dataset.column_names
# datasets = DatasetDict({'train': dataset})
train_dataset = dataset.map(
    prepare_train_features, 
    batched=True, 
    remove_columns=orig_col_names
)

In [13]:
print(len(train_dataset))
for example in train_dataset:
    print(len(example['input_ids']))

817
20
14
20
24
28
21
22
39
24
19
23
18
34
25
32
34
28
23
19
17
26
40
23
28
28
24
30
25
30
29
31
20
16
16
28
16
15
19
19
24
27
17
18
20
24
21
27
26
30
29
23
35
20
13
28
22
20
15
19
32
19
16
35
16
25
24
24
12
23
17
14
19
18
23
36
25
15
21
31
19
18
22
21
17
21
18
15
15
18
14
24
25
23
23
16
21
23
18
16
26
13
12
17
13
15
18
19
9
17
11
14
14
11
25
13
16
20
19
15
17
15
17
12
17
13
15
39
19
45
37
37
30
49
26
36
52
19
17
25
19
18
21
23
15
25
21
16
15
20
20
15
15
27
20
17
16
20
17
16
18
11
11
21
12
11
11
15
28
17
13
13
15
15
21
21
20
29
20
14
18
18
23
31
32
20
31
26
34
35
30
30
31
31
29
22
21
24
15
13
14
25
30
22
28
19
15
13
14
17
27
19
30
33
23
27
22
21
23
29
27
17
25
14
19
19
23
34
19
20
18
16
21
18
23
25
16
19
35
16
19
27
18
23
32
18
15
23
30
30
33
27
26
29
20
26
42
25
18
21
14
12
22
9
35
18
21
20
12
29
19
27
27
38
19
25
38
16
29
59
27
18
17
24
29
26
29
20
29
25
30
18
19
23
17
27
31
33
34
41
30
21
23
27
24
19
22
24
18
22
24
21
18
22
20
29
20
22
17
25
22
22
17
21
17
18
20
15
19
20
25
21
20
20

### 2.3 Run training 

In [17]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

batch_size = 128
args = TrainingArguments(
    f"test",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
    logging_steps = 100,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer, mlm=False,
)

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,5.420777
2,No log,6.068517
3,No log,6.333539
4,No log,6.486226
5,No log,6.698187
6,No log,6.863887
7,No log,6.940004
8,No log,6.987640
9,No log,6.989979
10,No log,6.980610


TrainOutput(global_step=70, training_loss=6.442421613420759, metrics={'train_runtime': 15.323, 'train_samples_per_second': 533.185, 'train_steps_per_second': 4.568, 'total_flos': 119698203082752.0, 'train_loss': 6.442421613420759, 'epoch': 10.0})

## 3. Evaluation

We're just going to evaluate the model on the same dataset it was trained on.